## ETL using Spark


## __Table of Contents__

<ol>
  <li>
    <a href="#Objectives">Objectives
    </a>
  </li>
  <li>
    <a href="#Datasets">Datasets
    </a>
  </li>
  <li>
    <a href="#Setup">Setup
    </a>
    <ol>
      <li>
        <a href="#Installing-Required-Libraries">Installing Required Libraries
        </a>
      </li>
      <li>
        <a href="#Importing-Required-Libraries">Importing Required Libraries
        </a>
      </li>
    </ol>
  </li>
  <li> 
    <a href="#Examples">Examples
    </a>
    <ol>
    <li>
      <a href="#Task-1---Create-a-Dataframe-from-the-raw-data-and-write-to-CSV-file.">Task 1 - Create a Dataframe from the raw data and write to CSV file.
      </a>
    </li>
    <li>
      <a href="#Task-2---Read-from-a-csv-file-and-write-to-parquet-file">Task 2 - Read from a csv file and write to parquet file
      </a>
    </li>
    <li>
      <a href="#Task-3---Condense-PARQUET-to-a-single-file.">Task 3 - Condense PARQUET to a single file.
      </a>
    </li>
    <li>
      <a href="#Task-4---Read-from-a-parquet-file-and-write-to-csv-file">Task 4 - Read from a parquet file and write to csv file
      </a>
    </li>
      </ol>
  

## Objectives

After completing this lab you will be able to:
 
 - Create a Spark Dataframe from the raw data and write to CSV file.
 - Read from a csv file and write to parquet file
 - Condense PARQUET to a single file.
 - Read from a parquet file and write to csv file


----


## Setup


For this lab, we will be using the following libraries:

*   [`PySpark`](https://spark.apache.org/docs/latest/api/python/index.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01) for connecting to the Spark Cluster


### Installing Required Libraries

Spark Cluster is pre-installed in the Skills Network Labs environment. However, you need libraries like pyspark and findspark to connect to this cluster.

If you wish to download this jupyter notebook and run on your local computer, follow the instructions mentioned <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/labs/Connecting_to_spark_cluster_using_Skills_Network_labs.ipynb">here.</a>



The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:


In [ ]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

### Importing Required Libraries

_We recommend you import all required libraries in one place (here):_


In [ ]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

from pyspark.sql import SparkSession


In [ ]:
#Create SparkSession
#Ignore any warnings by SparkSession command

spark = SparkSession.builder.appName("ETL using Spark").getOrCreate()

## Task 1 - Create a Dataframe from the raw data and write to CSV file.


In [ ]:
#create a list of tuples
#each tuple contains the student id, height and weight
data = [("student1",64,90),
        ("student2",59,100),
        ("student3",69,95),
        ("",70,110),
        ("student5",60,80),
        ("student3",69,95),
        ("student6",62,85),
        ("student7",65,80),
        ("student7",65,80)]

# some rows are intentionally duplicated

In [ ]:
#create a dataframe using createDataFrame and pass the data and the column names.

df = spark.createDataFrame(data, ["student","height_inches","weight_pounds"])

In [ ]:
# show the data frame

df.show()

Write to csv file

>**Note: In Apache Spark, when you use the write method to save a DataFrame to a CSV file, it indeed creates a directory rather than a single file. This is because Spark is designed to run in a distributed manner across multiple nodes, and it saves the output as multiple part files within a directory.The csv file is within the directory.**


In [ ]:
df.write.mode("overwrite").csv("student-hw.csv", header=True)

In [ ]:
#If you do not wish to over write use df.write.csv("student-hw.csv", header=True)

Verify the csv file


In [ ]:
# Load student dataset
df = spark.read.csv("student-hw.csv", header=True, inferSchema=True)

# display dataframe
df.show()

## Task 2 - Read from a csv file and write to parquet file


In [ ]:
# Load student dataset
df = spark.read.csv("student-hw.csv", header=True, inferSchema=True)

# display dataframe
df.show()

In [ ]:
# print the number of rows in the dataframe
df.count()

Drop Duplicates


In [ ]:
df = df.dropDuplicates()

In [ ]:
df.show()

In [ ]:
#Notice that the duplicates are removed

In [ ]:
# print the number of rows in the dataframe
df.count()

Drop Null values


In [ ]:
df=df.dropna()

In [ ]:
#Observe the rows with null values getting dropped
df.show()

Save to parquet file


In [ ]:
#Write the data to a Parquet file
df.write.mode("overwrite").parquet("student-hw.parquet")

In [ ]:
# if you do not wish to overwrite use the command df.write.parquet("student-hw.parquet")

In [ ]:
# verify that the parquet file(s) are created

In [ ]:
!!ls -l student-hw.parquet

Notice that there are a lot of .parquet files in the output.
- To improve parallellism, spark stores each dataframe in multiple partitions.
- When the data is saved as parquet file, each partition is saved as a separate file.


## Task 3 - Condense PARQUET to a single file.


Reduce the number of partitions in the dataframe to one.


In [ ]:
df = df.repartition(1)

Save to parquet file


In [ ]:
#Write the data to a Parquet file
df.write.mode("overwrite").parquet("student-hw-single.parquet")

In [ ]:
# if you do not wish to overwrite use the command df.write.parquet("student-hw-single.parquet")

In [ ]:
# verify that the parquet file(s) are created

In [ ]:
!ls -l student-hw-single.parquet

In [ ]:
#Notice that there is only one .parquet file

## Task 4 - Read from a parquet file and write to csv file


In [ ]:
df = spark.read.parquet("student-hw-single.parquet")

In [ ]:
df.show()

Transform the data


In [ ]:
#import the expr function that helps in transforming the data
from pyspark.sql.functions import expr

Convert inches to centimeters


In [ ]:
# Convert inches to centimeters
# Multiply the column height_inches with 2.54 to get a new column height_centimeters
df = df.withColumn("height_centimeters", expr("height_inches * 2.54"))
df.show()

Convert pounds to kilograms


In [ ]:
# Convert pounds to kilograms
# Multiply weight_pounds with 0.453592 to get a new column weight_kg
df = df.withColumn("weight_kg", expr("weight_pounds * 0.453592"))
df.show()

Drop the columns


In [ ]:
# drop the columns "height_inches","weight_pounds"
df = df.drop("height_inches","weight_pounds")
df.show()

Rename a column


In [ ]:
# rename the lengthy column name "height_centimeters" to "height_cm"
df = df.withColumnRenamed("height_centimeters","height_cm")
df.show()

Save to csv file


In [ ]:
df.write.mode("overwrite").csv("student_transformed.csv", header=True)

Verify the csv file


In [ ]:
# Load student dataset
df = spark.read.csv("student_transformed.csv", header=True, inferSchema=True)
# display dataframe
df.show()

Stop Spark Session


In [ ]:
spark.stop()

<!--
## Change Log
-->


<!--
|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2023-05-21|0.1|Ramesh Sannareddy|Initial Version Created|
-->
